<img src="https://www.mines.edu/webcentral/wp-content/uploads/sites/267/2019/02/horizontallightbackground.jpg" width="100%"> 

### CSCI250 Python Computing: Building a Sensor System
<hr style="height:5px" width="100%" align="left">

# Pulse Width Modulation

# Introduction

[Pulse width modulation](https://en.wikipedia.org/wiki/Pulse-width_modulation) is a process that generates square waves of specified frequency and duty cycle:
* the **frequency** is how often the signal repeats (given in Hz)
* the **duty cycle** is a percentage of high signal within a cycle.

<img src="https://cdn.sparkfun.com/assets/f/9/c/8/a/512e869bce395fbc64000002.JPG" width=40%>

# Simple PWM

Make a GPIO pin output high and low voltages in a regular pattern. 

In [4]:
import RPi.GPIO as GPIO
import time

# use GPIO numbering
GPIO.setmode(GPIO.BCM)

In [5]:
# simple PWM for set duration
def doPWM(pin, frequency, dutyCycle, duration):
        
    to = time.time()
    while time.time() < to + duration:
        
        GPIO.output(pin, True)              # output high
        time.sleep(   dutyCycle /frequency) # wait for a fraction of a cycle

        GPIO.output(pin, False)             # output low
        time.sleep((1-dutyCycle)/frequency) # wait for the remainder of the cycle

In [6]:
# Setup LED pin
ledPin = 13
GPIO.setup(ledPin, GPIO.OUT)

# PWM parameters
freq = 60      # [Hz] frequency
duty = 0.1   # [ ]  duty cycle
duration = 1  # [s]  duration

# Flash the LED
doPWM(ledPin, freq, duty, duration)

/tmp/ipykernel_12566/107472123.py:3: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(ledPin, GPIO.OUT)


<img src="http://www.dropbox.com/s/fcucolyuzdjl80k/todo.jpg?raw=1" width="10%" align="right">

Experiment by changing the frequency and the duty cycle.

### Problem with simple PWM

Connect the [buzzer](h_Buzzer.ipynb) and run the following code. 

It attempts to create a musical A note (440Hz). 

Notice that the signal is inconsistent, like a cracking voice. 

In [4]:
# Setup buzzer pin
buzPin = 19
GPIO.setup(buzPin, GPIO.OUT)

# PWM parameters
freq = 440    # [Hz] frequency
duty = 0.1  # [ ]  duty cycle
duration = 1  # [s]  duration

# Run buzzer
doPWM(buzPin, freq, duty, duration)

This problem is caused by the `time` module; it gets information from the OS kernel, which is slow with Python on an RPi. 

It is only accurate to a few ms, which is insufficient for playing musical notes at 100s or 1000s of Hz.

Measure `time.sleep()` by testing how long $1$ms takes. 
* It takes more than $1$ms on RPis: ~$2-10$ms.

For making music with a buzzer, we need something better.

In [47]:
t1 = time.time() # start timer
time.sleep(.001) # wait 1 ms
t2 = time.time() # stop timer

# print actual time taken
print((t2-t1)*1000,'ms')

2.4673938751220703 ms


In [48]:
# clean-up GPIO
GPIO.cleanup()

# Accurate PWM by `pigpio`

There are two options for creating more accurate PWM: 
* **software** timing with code written in C, or 
* **hardware** timing built-in on selected GPIO pins. 

The library [`pigpio`](http://abyz.me.uk/rpi/pigpio/python.html) provides access to both, plus many other useful features.

We cover only the most essential PWM features here.

### `pigpio` daemon
The `pigio` library requires its [daemon](https://en.wikipedia.org/wiki/Daemon_(computing)) to be running. 

To start the daemon, type the following command into the terminal:

[`sudo pigpiod -p 8887`](http://abyz.me.uk/rpi/pigpio/pigpiod.html)

**Restart the `pigpio` daemon every time you restart the RPi.**

Import the `pigpio` library and get a reference to the daemon.

In [4]:
# import pigpio library
import pigpio
import time

# create reference to pigpio daemon
pi = pigpio.pi(port = 8887)

## Hardware timing

Creates accurate PWM but is available just on **some** GPIO pins:

`12`, `13`, `18`, `19`.

Can generate **any** frequency.

[`pi.hardware_PWM(pin, frequency, duty_cycle)`](http://abyz.me.uk/rpi/pigpio/python.html#hardware_PWM) sets the frequency and duty cycle

* `pin` is the pin number written on the wedge
* `frequency`: integer in Hz
* `duty_cycle`: integer 0-1,000,000

Connect the buzzer and run the following code. 

It sounds much better than the code based on `time.sleep()`.

In [5]:
buzPin = 19

# PWM parameters
freq = 440    # [Hz] frequency
duty = 0.50   #      duty cycle
duration = 1  # [s]  duration

# turn the buzzer on
pi.hardware_PWM(buzPin, freq, int(duty * 1e6))

time.sleep(duration)

# turn the buzzer off
pi.hardware_PWM(buzPin, 0, 0)

0

## Software timing

Creates accurate PWM for **some** frequencies (Hz):

`8000, 4000, 2000, 1600, 1000, 800, 500, 400, 320, 250, 200, 160, 100, 80, 50, 40, 20, 10`.

Can be used on **any** GPIO pin.

[`pi.set_PWM_frequency(pin, frequency)`](http://abyz.me.uk/rpi/pigpio/python.html#set_PWM_frequency). 

[`pi.set_PWM_dutycycle(pin, duty_cycle)`](http://abyz.me.uk/rpi/pigpio/python.html#set_PWM_dutycycle). 

* `pin` is the pin number written on the wedge
* `frequency` is an integer in Hz
* `duty_cycle` is an integer that goes from 0-255

Connect the buzzer and run the following code. 

It sounds much better than the code based on `time.sleep()`.

In [7]:
buzPin = 19

# PWM parameters
freq = 440    # [Hz] frequency
duty = 0.50   #      duty cycle
duration = 5  # [s]  duration

# turn the buzzer on
pi.set_PWM_frequency(buzPin,freq)             # set frequency
pi.set_PWM_dutycycle(buzPin, int(duty * 255)) # set duty cycle

time.sleep(duration)

# turn the buzzer off
pi.set_PWM_dutycycle(buzPin, 0)

0

# Termination

Release the resources being used by `pigpio` using [`pi_stop()`](http://abyz.me.uk/rpi/pigpio/python.html#stop).

In [8]:
# release pigpio resources
pi.stop()

# PWM Uses

* **Buzzer**
    * Play tones for user feedback, music, alarms, etc.
    * Frequency has more noticeable effect than duty cycle
* **LED**
    * Create user feedback, indicate the state of something.
    * For fast frequencies, duty cycle affects brightness. 
    * Slower frequencies can create strobing effects.